In [2]:
install.packages("httr2")
library("httr2")
install.packages("jsonlite")
library("jsonlite") # Parse JSON
install.packages("listviewer")
library("listviewer") # View Json
library("tidyverse")
#install.packages("rjson")
#library("rjson")
install.packages(c("tm", "wordcloud","SnowballC"))

# Load libraries
library(tm)
library(wordcloud)
library(SnowballC)
library(rvest)

package 'httr2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\masip\AppData\Local\Temp\RtmpMvq0CA\downloaded_packages


Warning message:
"package 'httr2' was built under R version 4.4.2"


package 'jsonlite' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'jsonlite'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\00LOCK\jsonlite\libs\x64\jsonlite.dll to C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\jsonlite\libs\x64\jsonlite.dll: Permission denied"
Warning message:
"restored 'jsonlite'"



The downloaded binary packages are in
	C:\Users\masip\AppData\Local\Temp\RtmpMvq0CA\downloaded_packages


...

Warning message:
"cannot remove prior installation of package 'wordcloud'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\00LOCK\wordcloud\libs\x64\wordcloud.dll to C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\wordcloud\libs\x64\wordcloud.dll: Permission denied"
Warning message:
"restored 'wordcloud'"


package 'SnowballC' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'SnowballC'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\00LOCK\SnowballC\libs\x64\SnowballC.dll to C:\Users\masip\anaconda3\envs\mt4007\Lib\R\library\SnowballC\libs\x64\SnowballC.dll: Permission denied"
Warning message:
"restored 'SnowballC'"



The downloaded binary packages are in
	C:\Users\masip\AppData\Local\Temp\RtmpMvq0CA\downloaded_packages


Warning message:
"package 'tm' was built under R version 4.4.2"
Loading required package: NLP

Warning message:
"package 'NLP' was built under R version 4.4.2"

Attaching package: 'NLP'


The following object is masked from 'package:ggplot2':

    annotate


Warning message:
"package 'wordcloud' was built under R version 4.4.2"
Loading required package: RColorBrewer


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [ ]:

category<-"phy"
year<-"2022"
URI<-"http://api.nobelprize.org/2.1/nobelPrizes?limit=123&nobelPrizeYear=1901&yearTo=2024&nobelPrizeCategory=phy"
request <- request(URI)
response <- req_perform(request)
json <- response %>% resp_body_string
data<-jsonlite::fromJSON(json,flatten = TRUE)
#data<-data[1]
data<-bind_rows(data[[1]]["laureates"][[1]], .id = "column_label")
#print(jsonedit(json)) # View the JSON in  a nice way

#data <- fromJSON(json)
#data

In [61]:
#data[1]
#data[[1]]
#json
#class(data[[1]])
#class(data)
#class(data[[1]]["laureates"][[1]])
#colnames(data[[1]]["laureates"][[1]][[1]])
#data[[1]]["laureates"][[1]][1:122]
#data<-bind_rows(data[[1]]["laureates"][[1]], .id = "column_label")
#colnames(data)
motivations<-data$motivation.se
motivations<-str_replace_all(motivations, "[^\\w\\\\ ]"," ")
#str_replace_all()
class(motivations)

[1] "character"

In [62]:
writeLines(motivations,"data/motivations.txt")

In [63]:
mooncloud<-Corpus(DirSource("data"))
inspect(mooncloud)

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 2

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [64]:

# Strip unnecessary whitespace
mooncloud <- tm_map(mooncloud, stripWhitespace)
# Convert to lowercase
mooncloud <- tm_map(mooncloud, tolower)
# Remove conjunctions etc.
mooncloud <- tm_map(mooncloud, removeWords, stopwords("swedish"))
# Remove suffixes to the common 'stem'
mooncloud <- tm_map(mooncloud, stemDocument)
# Remove commas etc.
mooncloud <- tm_map(mooncloud, removePunctuation)

#(optional) arguments of 'tm' are converting the document to something other than text, to avoid, run this line
mooncloud <- tm_map(mooncloud, PlainTextDocument)

In [67]:
# Time to generate a wordcloud!
wordcloud(mooncloud
        , scale=c(5,0.5)     # Set min and max scale
        , max.words=100     # Set top n words
        , random.order=FALSE # Words in decreasing freq
        , rot.per=0.35       # % of vertical words
        , use.r.layout=FALSE # Use C++ collision detection
        , colors=brewer.pal(8, "Dark2"))

ERROR: Error in tolower(txt): non-character argument


In [ ]:
inspect(mooncloud)
class(mooncloud)

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 2

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 16357


[1] "SimpleCorpus" "Corpus"

# Webscraper


In [ ]:
BASE_URL<-"https://books.toscrape.com/catalogue/"
URL<-function(page=1){return(sprintf("https://books.toscrape.com/catalogue/page-%s.html",page))}
ht<-rvest::read_html(URL(2))



In [ ]:
df<-tibble(Rating=character(),Price=character(),URL=character())
for(i in 1:3){

ht<-rvest::read_html(URL(i))

prod_pod<-ht%>%html_elements(".product_pod")
rating<-prod_pod%>%html_element("p ") %>%html_attrs()
title<-prod_pod%>%html_element("a")%>%html_attrs()
price<-prod_pod%>%html_element(".price_color")%>%html_text()

rating<-str_extract(rating,pattern="(?<=star-rating ).\\w*")

book_url<-sprintf(paste0(BASE_URL,"%s"),as.character(title))
df<-df%>%add_row(tibble(Rating=rating,Price=price,URL=book_url))
}


Rating,Price,URL
<chr>,<chr>,<chr>
Three,£51.77,https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
One,£53.74,https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
One,£50.10,https://books.toscrape.com/catalogue/soumission_998/index.html
Four,£47.82,https://books.toscrape.com/catalogue/sharp-objects_997/index.html
Five,£54.23,https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
One,£22.65,https://books.toscrape.com/catalogue/the-requiem-red_995/index.html


In [ ]:
#df<-rbind(df,rating)
book_url<-df$URL
book_df<-tibble(URL=character(),UPC=character(),Title=character())
for(url in book_url){
    book_page<-rvest::read_html(url)
    title<-(book_page%>%html_elements("h1")%>%html_text2())[[1]]
    upc<-(book_page%>%html_elements("table")%>%html_table())[[1]][1,2][[1]]
    book_df<-book_df%>%add_row(URL=url,UPC=upc,Title=title)
    
}


In [ ]:
df<-df%>%left_join(book_df)%>%select(!URL)#%>%arrange(UPC,Title,Price,Rating)
df<-df%>%relocate(UPC,Title,Price,Rating)
#test_df%>%
test_list<-list(
"a897fe39b1053632",	
"90fa61229261140a",
"6957f44c3847a760",
"e00eb4fd7b871a48",
"4165285e1663650f")
df%>%filter(UPC %in% test_list)


Joining with `by = join_by(UPC, Title)`


UPC,Title,Price,Rating
<chr>,<chr>,<chr>,<chr>
a897fe39b1053632,A Light in the Attic,£51.77,Three
90fa61229261140a,Tipping the Velvet,£53.74,One
6957f44c3847a760,Soumission,£50.10,One
e00eb4fd7b871a48,Sharp Objects,£47.82,Four
4165285e1663650f,Sapiens: A Brief History of Humankind,£54.23,Five
